In [1]:
#Instalar el paquete que controla el navegador
!pip install selenium

#Descargar el Driver en :https://sites.google.com/a/chromium.org/chromedriver/downloads

#importar y chequer si selenium funciona

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait     
from selenium.webdriver.common.by import By     
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.select import Select
import openpyxl
from openpyxl import load_workbook
import re
import itertools
import time 
import pandas as pd

# selecionando nuestro webbrowser (selecionar la ubicación donde este el driver)
try:
    driver = webdriver.Chrome("C:/Users/Ivan/Documents/CCCE/WebDriver/chromedriver80")
except:
    driver = webdriver.Chrome("C:/Users/Ivan/Documents/CCCE/WebDriver/chromedriver85")
    print("El driver fue instalado correctamente")

El driver fue instalado correctamente


In [2]:
#pagina donde va a buscar
#Dafiti
driver.maximize_window()

docs = openpyxl.load_workbook ("Categorias.xlsx")
hoja = docs.get_sheet_by_name("Dafiti")

itemsD = []
for columna in hoja.iter_rows():
    bien = columna[0].value
    itemsD.append(bien)

<ipython-input-2-41c9a50908b2>:6: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  hoja = docs.get_sheet_by_name("Dafiti")


In [3]:
driver.get("https://www.dafiti.com.co/")

In [5]:
linksD = []
priceD = []
descD = []
codesD = []
timeD = []
obj = []
timeDF = []

from selenium.webdriver.common.keys import Keys

for item in itemsD:
    searchBar = WebDriverWait(driver, 200).until(lambda x: x.find_element_by_id("searchInput")) #Selecionar la barra de busqueda
    searchBar.send_keys(item) #escribir en la barra de busqueda
    searchBar.send_keys(Keys.RETURN) #Enviar la busqueda sortNumber1
    print(item)
    time.sleep(20)
    try:
        sort = WebDriverWait(driver, 60).until(lambda x: x.find_element_by_css_selector(".itm.hasOverlay.lfloat.size1of3.catalog-discount-item"))
        sort.click()
    except:
        sort2 = WebDriverWait(driver, 60).until(lambda x: x.find_element_by_css_selector(".specialPriceText"))
        sort2.click()
    time.sleep(5)
    aplicar = WebDriverWait(driver, 60).until(lambda x: x.find_element_by_css_selector(".filterBtn.applyFilterButton.greenButton"))
    aplicar.click()
    time.sleep(20)
    elemsp = WebDriverWait(driver, 200).until(lambda x: x.find_elements_by_css_selector(".itm-price.special"))
    for elemp in elemsp:
        if elemp.text != "":
            ele = elemp.text #obtiene el precio del webelement
            value = re.findall(r"[0-9.]*[0-9]+", ele) #quita el signo "$" del precio
            #print(value[0])
            priceD.append(value[0]) #guarda el precio en una lista
        else:
            continue
    elems = WebDriverWait(driver, 60).until(lambda x: x.find_elements_by_css_selector(".itm-link"))
    for elem in itertools.islice(elems,None,None,1): #realiza un loop cada "3" Webelements
        linksD.append(elem.get_attribute("href")) #Obtiene y guarda el webelement en la lista "linksK"
        #print(elem.get_attribute("href"))
        timeD.append(time.strftime("%d/%m/%y %H:%M:%S"))
        timeDF.append(time.strftime("%d/%m/%y"))
        obj.append(item)
    #print(len(priceD))
    #print(len(linksD))
for link in linksD: #Obtener descripcion & Codigo unico de los links
    des = re.findall(r"co?/([^\s].+)-", link)
    des = des[0]
    #print(des)
    descD.append(des) #guarda cada descripcion en la lista "descK"
    code = re.findall("-([0-9].+).html",link)
    code = code[0]
    #print(code)
    codesD.append(code) #guarda cada descripcion en la lista "codesK"

Camisa hombre


TimeoutException: Message: 


In [19]:
nowa = time.strftime("%d-%m-%y")
dataf = pd.DataFrame({"Codigo":codesD, "Descripción":descD,
                      "Subcategoria":obj, "Precio":priceD,
                      "Link":linksD, "Fecha & Hora":timeD, "Fecha":timeDF})
book = load_workbook('Dafiti.xlsx')
writer = pd.ExcelWriter('Dafiti.xlsx', engine = 'openpyxl')
writer.book = book
dataf.to_excel(writer, sheet_name = ("Dafiti " + str(nowa)), index = False)
writer.save()

In [7]:
len(priceD)

666

In [8]:
len(linksD)

672

In [17]:
priceD = []
bienes = WebDriverWait(driver, 60).until(lambda x: x.find_elements_by_css_selector(".itm-product-main-info"))
for bien in bienes:
    pric = bien.find_elements_by_css_selector(".itm-price")
    for p in pric[0:1]:
            ele = p.text #obtiene el precio del webelement
            value = re.findall(r"[0-9.]*[0-9]+", ele) #quita el signo "$" del precio
            print(value)
            priceD.append(value) #guarda el precio en una lista

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [12]:
len(priceD)

36

In [1]:
import dropbox
from dropbox.files import WriteMode

cliente = dropbox.Dropbox('sl.Ak9Xgcw8jKOKY9oqvWvcLIKykrRLgbZdFJ8A-ERmUjoYSzyz9f90Ae6ZybmJ_f5EhvscXuicOXQpB0ao6E2FiiH85TdKg3tqfIaYTUac12-rMvospyfQ-obONJULytZK2v6l4dxYH_Y')
path = "/Dropbox/Aplicaciones/save_python/Dafiti.xlsx"
#print(cliente.users_get_current_account()) 
f= open("C:/Users/Ivan/Documents/CCCE/Dafiti/Dafiti.xlsx", "rb")
respuesta = cliente.files_upload(f.read(),path, mode = dropbox.files.WriteMode.overwrite)